In [1]:
import os

%pwd

'/home/tousside/Documents/recrutement/cowrywise-customer-plan-abandonment/research'

os.chdir("../")

%pwd



'/home/ubuntu/africlimateai/rainfall-prediction'

In [44]:


from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    all_schema: str
    test_size: float



In [45]:


from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [46]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: str = CONFIG_FILE_PATH,
                 params_filepath: str = PARAMS_FILE_PATH,
                 schema_filepath: str = SCHEMA_FILE_PATH,
                 ):
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                self.schema = read_yaml(schema_filepath)
                create_directories([self.config.artifacts_root])
                

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        schema = self.schema
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            all_schema=schema,
            test_size=config.test_size
            
        )
        
        return data_transformation_config

In [47]:

import os 
from mlProject import logger
import pandas as pd

In [61]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config
        
    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)
        features = list(self.config.all_schema.COLUMNS.keys())
        target = list(self.config.all_schema.TARGET_COLUMN.keys())
        all_variables = features + target
        data = data[all_variables]
        
        data["time"] = pd.to_datetime(data.time)
        
        data_size = len(data)
        test_size = int(self.config.test_size*data_size)
        train_size = data_size-test_size
        train_set = data[:train_size]
        test_set = data[train_size:]

        
        train_set.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test_set.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train_set.shape)
        logger.info(test_set.shape)



In [62]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-09-23 15:12:58,172: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-23 15:12:58,173: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-23 15:12:58,176: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-23 15:12:58,176: INFO: common: created directory at: artifacts]
[2025-09-23 15:12:58,178: INFO: common: created directory at: artifacts/data_transformation]
[2025-09-23 15:12:58,192: INFO: 2321397177: Splited data into training and test sets]
[2025-09-23 15:12:58,193: INFO: 2321397177: (187, 12)]
[2025-09-23 15:12:58,193: INFO: 2321397177: (80, 12)]
